In [33]:
# Video: https://www.youtube.com/watch?v=2JDR6jv0fGA
import pandas as pd
import numpy as np

seasons = [str(season) for season in range(2013, 2023)]

full_team_names = {
    'crd': 'Arizona Cardinals', 'atl': 'Atlanta Falcons', 'rav': 'Baltimore Ravens', 'buf': 'Buffalo Bills', 'car': 'Carolina Panthers', 
    'chi': 'Chicago Bears', 'cin': 'Cincinnati Bengals', 'cle': 'Cleveland Browns', 'dal': 'Dallas Cowboys', 'den': 'Denver Broncos', 
    'det': 'Detroit Lions', 'gnb': 'Green Bay Packers', 'htx': 'Houston Texans', 'clt': 'Indianapolis Colts', 'jax': 'Jacksonville Jaguars', 
    'kan': 'San Diego Chargers', 'sdg': 'Los Angeles Chargers', 'ram': 'Los Angeles Rams', 'rai': 'Las Vegas Raider', 'mia': 'Miami Dolphins', 
    'min': 'Minnesota Vikings', 'nwe': 'New England Patriots', 'nor': 'New Orleans Saints', 'nyg': 'New York Giants', 'nyj': 'New York Jets', 
    'phi': 'Philadelphia Eagles', 'pit': 'Pittsburgh Steelers', 'sea': 'Seattle Seahawks', 'sfo': 'San Francisco 49ers', 'tam': 'Tampa Bay Buccaneers', 'oti': 'Tennessee Titans', 'was': 'Washington Redskins'
}
teams = list(full_team_names.keys())
print(len(seasons))

10


In [43]:
import random
import time
import os

nfl_games_df = pd.DataFrame()
folder_path = 'csv_files'
file_name = 'gamelogs_2013-2023.csv'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

file_path = os.path.join(folder_path, file_name)

if not os.path.exists(file_path):
    for season in seasons:
        for team in teams:
            # getting the gamelog for every team in every year
            url = "https://www.pro-football-reference.com/teams/" + team + "/" + season + "/gamelog/"
            print(url)
            # https://www.pro-football-reference.com/teams/sdg/2023/gamelog/
            off_df = pd.read_html(url, header=1, attrs={'id':'gamelog' + season})[0]
            def_df = pd.read_html(url, header=1, attrs={'id':'gamelog_opp' + season})[0]
            team_df = pd.concat([off_df, def_df], axis=1)
            
            team_df.insert(loc=0, column='Season', value=season)
            team_df.insert(loc=2, column='Team', value=full_team_names[team])
            
            nfl_games_df = pd.concat([nfl_games_df, team_df], ignore_index=True)
            
            time.sleep(random.randint(4, 5))
        
        print(nfl_games_df)
    nfl_games_df.to_csv(file_path, index=False)
else:
    nfl_games_df = pd.read_csv(file_path)

In [44]:
# Cleans up the dataframe, makes sure to name some columns
new_columns = {'Unnamed: 4': 'Win', 'Unnamed: 6':'Home', 'Tm':'Off_Pts', 'Opp.1':'Def_Pts'}
nfl_games_df = nfl_games_df.rename(columns=new_columns)

In [46]:
# Replacing values in columns for Win and Home
nfl_games_df['Win'] = nfl_games_df['Win'].apply(lambda x: 1 if x == 'W' else(0 if x == 'L' else 0.5))
nfl_games_df['Home'] = nfl_games_df['Home'].apply(lambda x: 0 if x == '@' else 1)

# Replacing values in the OT column of the dataframe
nfl_games_df['OT'] = nfl_games_df['OT'].apply(lambda x: 1 if x == 'OT' else 0)

In [ ]:
# Get rid of Unnamed: 3, index 5, 6th item in table
# Can get consider getting rid of OT
# Can get rid of everything after opponent
# Need to merge with team rankings and opponent rankings